# Algorytm sympleks

### Programowanie liniowe

Jest to metoda służąca osiągnięciu jak najlepszego rozwiązania w procesie podejmowania decyzji. Programowanie liniowe jest najczęściej stosowanym modelem optymalizacji ze względu na sprawny algorytm obliczeń, a także dzięki możliwości efektownego przedstawienia zagadnień w postaci graficznej. Wadą natomiast jest to, iż nie wszystko można wyrazić za pomocą liczb.

W skład modelu będącego zadaniem programowania liniowego wchodzą:
1. <b>funkcja celu</b> - obrazuje cel, który ma zostać osiągnięty,
2. <b>zmianne decyzyjne</b> - określa narzędzia i zasoby wykorzystane w celu osiągnięcia celu,
3. <b>warunki ograniczające</b> - opisują przeszkody, które mogą pojawić się w trakcie realizacji celu.

Jeżeli model posiada dwie zmienne, łatwo jest go rozwiązać metodą geometryczną w układzie współrzędnych kartezjańskich. Natomiast w przypadku większej liczby zmiennych decyzyjnych, sposób ten się nie sprawdzi. W takiej sytuacji skorzystać można z innej techniki, jaką jest metoda sympleks.

### Metoda sympleks

Nazwa nawiązuje do <b>sympleksu</b>: k-wymiarowym sympleksem nazywamy k-wymiarowy wielościan, który jest otoczką wypukłą swoich k+1 wierzchołków.

![alt text](sympleks.png "Title")

Zadanie programowania liniowego z dowolną liczbą zmiennych można rozwiązać, wyznaczając wszystkie wierzchołkowe punkty wielościanu, a następnie porównując wartości funkcji celu w punktach wierzchołkowych.

W metodzie sympleks można wyróżnić dwa kroki:

<b>Krok 1.</b>
Sprowadzenie warunków ograniczających do układu równań liniowych.

Np. poniższe ograniczenie:
$$2x_{1} + 7x_{2} \leq 13$$

przekształcamy do postaci: $$2x_{1} + 7x_{2} + x_{3} = 13.$$

<b>Krok 2.</b> 
Obliczanie zmiennych decyzyjnych, które przedstawiają najlepszą decyzję dostosowaną do ustalonego modelu.

W związku z wielością punktów powstaje problem wyznaczenia wartości funkcji celu i znalezienie optymalnego wierzchołka, który spełniłby warunek zadania programowania liniowego. Rozwiązanie tego problemy sprowadza się do tego, że znając dowolny wierzchołkowy punkt i wartość funkcji celu w tym punkcie, odrzuca się wszystkie wierzchołki, w których funkcja ta przyjmuje gorsze wartości

W kolejnym kroku, z obecnego wierzchołka, należy przejść krawędzią do sąsiadującego z nim wierzchołka, w którym funkcja celu osiąga lepsze wartości.
Iteracja kończy się w momencie, gdy kolejny przeglądany wierzchołek jest najlepszy pod względem odpowiednich wartości funkcji celu.

![alt text](simplex_alg.png "Title")

### Zastosowania

W działalność gospodarczej, czy też zarządzaniu produkcją dąży się do maksymalnego wykorzystania dostępnych środków w procesie realizacji założonego celu. Zasada ta sprowadza się do zagadnień optymalizacji oraz zasady największej efektywności. Jeżeli z określonych środków osiągany jest maksymalny nakład lub określony cel jest zrealizowany przy jak najmniejszych nakładach, możemy mówić o planie optymalnym. 

Konkretnym przykładem wykorzystania metody sympleks może być problem transportowy. Polega on na znalezieniu takiej drogi od dostawcy towaru do odbiorcy, aby poniesione koszta były jak najmniejsze.

### Implementacja

In [2]:
import numpy as np

In [3]:
#modywikacja danych 
def gen_matrix(var,cons):    
    tab = np.zeros((cons+1, var+cons+2))    
    return tab

def convert(eq):
    eq = eq.split(',')
    if 'G' in eq:
        g = eq.index('G')
        del eq[g]
        eq = [float(i)*-1 for i in eq]
        return eq
    if 'L' in eq:
        l = eq.index('L')
        del eq[l]
        eq = [float(i) for i in eq]
        return eq

def add_cons(table):
    lr = len(table[:,0])
    empty = []
    for i in range(lr):
        total = 0
        for j in table[i,:]:                       
            total += j**2
        if total == 0: 
            empty.append(total)
    if len(empty)>1:
        return True
    else:
        return False

def constrain(table,eq):
    if add_cons(table) == True:
        lc = len(table[0,:])
        lr = len(table[:,0])
        var = lc - lr -1      
        j = 0
        while j < lr:            
            row_check = table[j,:]
            total = 0
            for i in row_check:
                total += float(i**2)
            if total == 0:                
                row = row_check
                break
            j +=1
        eq = convert(eq)
        i = 0
        while i<len(eq)-1:
            row[i] = eq[i]
            i +=1        
        row[-1] = eq[-1]
        row[var+j] = 1    
    else:
        print('Cannot add another constraint.')

In [4]:
def next_round_r(table):    
    m = min(table[:-1,-1])    
    if m>= 0:        
        return False    
    else:        
        return True
        
def next_round(table):    
    lr = len(table[:,0])   
    m = min(table[lr-1,:-1])    
    if m>=0:
        return False
    else:
        return True

In [5]:
def find_neg_r(table):
    lc = len(table[0,:])
    m = min(table[:-1,lc-1])
    if m<=0:        
        n = np.where(table[:-1,lc-1] == m)[0][0]
    else:
        n = None
    return n

def find_neg(table):
    lr = len(table[:,0])
    m = min(table[lr-1,:-1])
    if m<=0:
        n = np.where(table[lr-1,:-1] == m)[0][0]
    else:
        n = None
    return n

In [6]:
def loc_piv_r(table):
    total = []        
    r = find_neg_r(table)
    row = table[r,:-1]
    m = min(row)
    c = np.where(row == m)[0][0]
    col = table[:-1,c]
    for i, b in zip(col,table[:-1,-1]):
        if i**2>0 and b/i>0:
            total.append(b/i)
        else:                
            total.append(10000)
    index = total.index(min(total))        
    return [index,c]
    
def loc_piv(table):
    if next_round(table):
        total = []
        n = find_neg(table)
        for i,b in zip(table[:-1,n],table[:-1,-1]):
            if b/i >0 and i**2>0:
                total.append(b/i)
            else:
                total.append(10000)
        index = total.index(min(total))
        return [index,n]

In [7]:
def pivot(row,col,table):
    lr = len(table[:,0])
    lc = len(table[0,:])
    t = np.zeros((lr,lc))
    pr = table[row,:]
    if table[row,col]**2>0:
        e = 1/table[row,col]
        r = pr*e
        for i in range(len(table[:,col])):
            k = table[i,:]
            c = table[i,col]
            if list(k) == list(pr):
                continue
            else:
                t[i,:] = list(k-r*c)
        t[row,:] = list(r)
        return t
    else:
        print('Cannot pivot on this element.')

In [8]:
def convert_min(table):
    table[-1,:-2] = [-1*i for i in table[-1,:-2]]
    table[-1,-1] = -1*table[-1,-1]    
    return table

In [9]:
def gen_var(table):
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    v = []
    for i in range(var):
        v.append('x'+str(i+1))
    return v

In [10]:
def add_obj(table):
    lr = len(table[:,0])
    empty = []
    for i in range(lr):
        total = 0        
        for j in table[i,:]:
            total += j**2
        if total == 0:
            empty.append(total)    
    if len(empty)==1:
        return True
    else:
        return False

In [11]:
def obj(table,eq):
    if add_obj(table)==True:
        eq = [float(i) for i in eq.split(',')]
        lr = len(table[:,0])
        row = table[lr-1,:]
        i = 0        
        while i<len(eq)-1:
            row[i] = eq[i]*-1
            i +=1
        row[-2] = 1
        row[-1] = eq[-1]
    else:
        print('You must finish adding constraints before the objective function can be added.')

In [12]:
def maxz(table):
    while next_round_r(table)==True:
        table = pivot(loc_piv_r(table)[0],loc_piv_r(table)[1],table)
    while next_round(table)==True:
        table = pivot(loc_piv(table)[0],loc_piv(table)[1],table)        
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]            
            val[gen_var(table)[i]] = table[loc,-1]
        else:
            val[gen_var(table)[i]] = 0
    val['max'] = table[-1,-1]
    return val

In [13]:
def minz(table):
    table = convert_min(table)
    while next_round_r(table)==True:
        table = pivot(loc_piv_r(table)[0],loc_piv_r(table)[1],table)    
    while next_round(table)==True:
        table = pivot(loc_piv(table)[0],loc_piv(table)[1],table)       
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]             
            val[gen_var(table)[i]] = table[loc,-1]
        else:
            val[gen_var(table)[i]] = 0 
            val['min'] = table[-1,-1]*-1
    return val

In [14]:
m = gen_matrix(2,2)
constrain(m,'2,-1,G,10')
constrain(m,'1,1,L,20')
obj(m,'5,10,0')
print(maxz(m))     
m = gen_matrix(2,4)
constrain(m,'2,5,G,30')
constrain(m,'-3,5,G,5')
constrain(m,'8,3,L,85')
constrain(m,'-9,7,L,42')
obj(m,'2,7,0')
print(minz(m))

{'x1': 10.0, 'x2': 10.0, 'max': 150.0}
{'x1': 5.0, 'x2': 4.0}


# Zadanie programowania liniowego

![alt text](ex.png "Title")

In [15]:
from gilp.simplex import LP
from gilp.visualize import simplex_visual

A = np.array([[0, 1],
              [1, -1],
              [1, 0],
              [-2, 1]])
b = np.array([[4],
              [2],
              [3],
              [0]])
c = np.array([[1],
              [2]])

lp = LP(A,b,c)

simplex_visual(lp).show()

Typy przechodzenia między wierzchołkami:<br>
Największy Wzrost - wymieniona wyżej, wybiera wierzchołek z najwyższym zyskiem<br>
Zasada Blanda - po najmniejszym indeksie. Wyklucza cykle. W teorii przydatna, w praktyce zbyt wolna <br>
Zasada Dantziga - po najbardziej zredukowanym koszcie zmiennej nie bazowej. Nazwana po twórcy metody sympleks.<br>

In [16]:
from gilp import examples as ex

x = np.array([[0],[0],[0],[5],[25],[125]])
simplex_visual(ex.KLEE_MINTY_3D_LP, initial_solution=x, rule='greatest_ascent').show()

In [17]:
from gilp import examples as ex

x = np.array([[0],[0],[0],[5],[25],[125]])
simplex_visual(ex.KLEE_MINTY_3D_LP, initial_solution=x, rule='bland').show()

In [18]:
from gilp import examples as ex

x = np.array([[0],[0],[0],[5],[25],[125]])
simplex_visual(ex.KLEE_MINTY_3D_LP, initial_solution=x, rule='dantzig').show()

Metoda sympleksu posiada podobne ograniczenia do metody spadku gradientu. <br> 
- W zależności od początkowego wierzchołka metoda sympleks może osiągnąc maksimum lokalne a nie globalne. <br>
- Procedura może nie znaleźć rozwiązania. <br>
- Nie ma dobrego graficznego sposobu na stwierdzenie czy metoda osiągnęła maksimum globalne. <br>
- Procedura nie może zostać urównoleglona w systemach rozproszonych. <br>
- Nie można dołożyc ograniczeń po wykonaniu metody.